In [1]:
import mysql
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config # Read database configuration file and return a dictionary database object

In [2]:
def create_msql_database(name_database):
    """Function create MYSQL database to save into db config file"""
    mydb = mysql.connector.connect(host='localhost',
                                      user='root',
                                      password='root')
    
    mycursor = mydb.cursor()
    mycursor.execute("SHOW DATABASES")
    databases = [x[0] for x in mycursor]
    if name_database not in databases:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {name_database}")
    else:
        print("Database exists")
    mycursor.close()
    
def create_msql_table(query_create):
    """
    Execute query to create table
    :query_create:: to create table
    """

    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')
        cursor = conn.cursor()
        print('Creating Mysql table...')
        cursor.execute(query_create)
        print('Table created')
    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()

def insert_msql_table(file_dir, table_name):
    """execute query to insert tsv file into table
    :file_dir:: file path
    :table_name:: name of table to be insterted
    :skip_header:: use header 
    """

    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')

        cursor = conn.cursor()
        print('creating MYSQL table...')
        load_data_query = f"LOAD DATA INFILE '{file_dir}' INTO TABLE {table_name}\
                                FIELDS TERMINATED BY '\t'\
                                LINES TERMINATED BY '\n'\
                                IGNORE 1 LINES\
                                (@net_id, @net_name, @cidr, @net_range, @net_type, @asn, @org_cust_id, @updated)\
                                SET net_id = NULLIF(@net_id, ' '), net_name = NULLIF(@net_name, ' '), cidr = NULLIF(@cidr, ' '), net_range = NULLIF(@net_range, ' '), net_type = NULLIF(@net_type, ' '), asn = NULLIF(@asn, ' '), org_cust_id = NULLIF(@org_cust_id, ' '), updated = NULLIF(@updated, ' ')\
                                ON DUPLICATE KEY UPDATE net_name = VALUES(net_name), cidr = VALUES(cidr), net_range = VALUES(net_range), net_type = VALUES(net_type), asn = VALUES(asn), org_cust_id = VALUES(org_cust_id), updated = VALUES(updated);"


        cursor.execute(load_data_query)

        conn.commit()
        print('MYSQL table created')

    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()



In [3]:

# Create the table (if it doesn't exist)
network_table = "CREATE TABLE network (net_id VARCHAR(255) UNIQUE, \
                                            net_name VARCHAR(255), cidr VARCHAR(255) UNIQUE, \
                                                net_range VARCHAR(255), net_type VARCHAR(255), \
                                                    asn VARCHAR(255), org_cust_id VARCHAR(255), updated VARCHAR(255));"
organization_table = "CREATE TABLE organization (org_id VARCHAR(255) UNIQUE, \
                                                            org_name VARCHAR(255), updated VARCHAR(255));"
customer_table = "CREATE TABLE customer (cust_id VARCHAR(255) UNIQUE, \
                                                            cust_name VARCHAR(255), updated VARCHAR(255));"

In [4]:
########
create_msql_database('ThinkCX')
create_msql_table(network_table)
#create_msql_table(organization_table)
#create_msql_table(customer_table)

Database exists
Connecting to MYSQL database...
Connected to MYSQL database
Creating Mysql table...
Error:  1050 (42S01): Table 'network' already exists


In [5]:
insert_msql_table('/Users/hople/working_folder/Bootcamp_practices/SQL_practice/arin_data_sets/arin_net.tsv', 'network')

Connecting to MYSQL database...
Connected to MYSQL database
creating MYSQL table...
Error:  1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'ON DUPLICATE KEY UPDATE net_name = VALUES(net_name), cidr = VALUES(cidr), net_ra' at line 2
